- ChatPromptTemplate is to create a template from messages.
- prompt template is to create a template from string.

In [1]:
#from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# streaming means we see the response of the model as it happens instead of just waiting till the end.

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]) # -> gpt-3.5 turbo model
   

In [4]:
template = PromptTemplate.from_template("what is the distance between {country_a} and {country_b}")
#llm = OpenAI() -> text-davinci model
    # temperature determines how creative or random the model is.

prompt = template.format(country_a = "Korea", country_b = "Thailand")
chat.predict(prompt)

The distance between Korea and Thailand is approximately 3,000 kilometers (1,864 miles).

'The distance between Korea and Thailand is approximately 3,000 kilometers (1,864 miles).'

#### predict message using gpt-3.5 turbo model

In [5]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

message = [
    SystemMessage(content="You are a geography expert. And you only reply in Italian."),
    AIMessage(content="Ciao, mi ciamo Paulo."),
    HumanMessage(content="What is the distance between Mexico and Thailand? Also, What is your name? ")
]

chat.predict_messages(message)

Ciao! Il mio nome è Paolo. La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri.

AIMessageChunk(content='Ciao! Il mio nome è Paolo. La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri.')

#### Now, we will use ChatPromptTemplate to make it simpler.
- template -> format -> predict method

In [6]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "My name is {name}."),
        ("human", "What is the distance between {country_a} and {country_b}? Also, What is your name?")
    ]
)
 
prompt = template.format_messages(
    language = "English",
    name = "Joong Ho",
    country_a = "Korea",
    country_b = "Thailand"
)

chat.predict_messages(prompt)

The distance between Korea and Thailand varies depending on the specific locations. The approximate distance between Seoul, South Korea, and Bangkok, Thailand, is around 4,500 kilometers (2,800 miles). However, please note that this is a straight-line distance and actual travel distances may differ.

As for my name, I am an AI language model developed by OpenAI, and I don't have a personal name. You can simply refer to me as "Assistant." How can I assist you further?

AIMessageChunk(content='The distance between Korea and Thailand varies depending on the specific locations. The approximate distance between Seoul, South Korea, and Bangkok, Thailand, is around 4,500 kilometers (2,800 miles). However, please note that this is a straight-line distance and actual travel distances may differ.\n\nAs for my name, I am an AI language model developed by OpenAI, and I don\'t have a personal name. You can simply refer to me as "Assistant." How can I assist you further?')

## OutputParser and LangChain Expression Language(LCEL)

- Need Output Parser to transform the response from LLM
- It could be List, Dictionary, or database. 

In [7]:
from langchain.schema import BaseOutputParser 

class CommaOutputParser(BaseOutputParser):
    
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip,items))

# p = CommaOutputParser()
# p.parse("Hello, How, are,you?")

In [8]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply iwth anything else.",
     ),
    ("human", "{question}")
    ]
)

prompt = template.format_messages(
    max_items="10",
    question="What are the colors?"
    )

#chat.predict_messages(prompt)
result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

red, orange, yellow, green, blue, indigo, violet, black, white, gray

['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

- Make above cell very simple and short by creating "chain".
- Need Template, Output Parser, and Chat Model.


In [9]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply iwth anything else.",
     ),
    ("human", "{question}")
    ]
)

# this chain combines template, language model and Output Parser
chain = template | chat | CommaOutputParser()

# this is example. you can also chain among chains
# chain_one = template | chat | CommaOutoputParser()
# chain_two = template2 | chat | OutputParser()
# first, combine chain_one, and you can use output of chain_one to use it with chain_two
# all = chain_one | chain_two| output

chain.invoke({
    "max_items":5,
    "question":"What are Pokemons?"
})

pikachu, charizard, bulbasaur, squirtle, jigglypuff

['pikachu', 'charizard', 'bulbasaur', 'squirtle', 'jigglypuff']

### Chaining Chains
- https://python.langchain.com/docs/expression_language/interface


In [ ]:
# dummy example
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", '''You are a world-class international chef.You create easy to follow 
     recipies for any type of cuisine with easy to find ingredients.'''),
    ("human", "I want to cook {cuisine} food"),   
])

# create chef chain then send chat to LLM
# we are going to receive recipe from a chef, and we will adapt it to use vegetarian ingredients.
chef_chain = chef_prompt | chat

In [ ]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", '''You are a vegetarian chef specialized on making traditional recipies
     vegetarian. You find alterative ingredients and explain their preparation. You
     don't radically modify the recipe. If there is no alternative for a food just 
     say you don't know how to replace it.'''),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

# final_chain = chef_chain | veg_chain
# final_chain.invoke({
#    "cuisine": "indian"  
# })
# veg_chain.invoke({
#     "recipe": "chatmodel"
# })


# Instead of calling two invoke function, input called recipe of the veg_chain come from chef_chain
# putting context obejct for the chain
# store the result of chef_chain on a key called "recipe"
# give this recipe key to the veg_chain
final_chain = {"recipe": chef_chain} | veg_chain
final_chain.invoke({"cuisine": "indian"})


AIMessage(content="To make a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based protein such as tofu or tempeh. Here's how you can modify the recipe:\n\nIngredients:\n- 500g tofu or tempeh, cut into bite-sized pieces\n- 1 cup plain yogurt (use dairy-free yogurt for a vegan version)\n- 2 tablespoons lemon juice\n- 2 tablespoons vegetable oil\n- 1 large onion, finely chopped\n- 3 cloves of garlic, minced\n- 1 tablespoon ginger, grated\n- 2 teaspoons ground cumin\n- 2 teaspoons ground coriander\n- 1 teaspoon turmeric powder\n- 1 teaspoon paprika\n- 1 teaspoon garam masala\n- 1 cup tomato puree\n- 1 cup coconut cream (or dairy-free heavy cream)\n- Salt, to taste\n- Fresh cilantro, chopped (for garnish)\n\nInstructions:\n1. In a bowl, combine the yogurt, lemon juice, 1 teaspoon of ground cumin, 1 teaspoon of ground coriander, turmeric powder, paprika, and salt. Mix well.\n2. Add the tofu or tempeh pieces to the marinade and coat them evenly. Let it ma